# Compare sessions for same subjects acquired from same field strength 
 - Questions:
    - How do maps compare between sessions? [help understand differences between 3T and 7T, are they due to raw data? If due to raw data then would expect]
    - What is the reliability of MR features between sessions? How do field strength differences compare to differences expected by repeated acquisitions?
    - Is this reliability moderated by sex/gender, time between sessions, patient grouping, MR strength?
 - Outputs of interest:
    - Subj by vertex matrix plots with columns being different sessions and rows having matched subject-MR strength combos
    - Correlation between vertex values between sessions

Steps:    
0. CSV file with all participants and paths to smoothed data (see main.ipynb)
1. Make one df with list of all participant-features available per session number
1. Compute statistics on these dfs (cor between rows with the same ID-study)
1. Plot matrices


In [93]:
import os
import sys
import pandas as pd
import numpy as np

import importlib
import datetime
import matplotlib.pyplot as plot
import pickle

sys.path.append(os.getcwd())
#sys.path.append("/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code")
sys.path.append("/host/verges/tank/data/daniel/")

import tTsTGrpUtils as tsutil
importlib.reload(tsutil)

import demo
importlib.reload(demo)

#from genUtils import id, gen, t1

<module 'demo' from '/host/verges/tank/data/daniel/3T7T/z/code/analyses/demo.py'>

In [94]:
# import csv, created from demographics processing (see main.ipynb)
# run parameters

# details of demographics file
demographics = { # should include UID
    "pth" : "/host/verges/tank/data/daniel/3T7T/z/outputs/03b_demoPths_clean_16Sep2025-104025.csv",
    # column names:
    'nStudies': True, # whether multiple studies are included
    "ID_7T" : "PNI_ID", 
    "ID_3T" : "MICS_ID",
    "SES" : "SES",
    "date": "Date", # scan date
    "age": "age",
    "sex": "sex",
    "grp" : "grp_detailed" # col name for participant grouping variable to use
}

# specify root directories
MICs = {
    "name": "MICs",
    "dir_root": "/data/mica3/BIDS_MICs",
    "dir_raw": "/rawdata",
    "dir_deriv": "/derivatives",
    "dir_mp": "/micapipe_v0.2.0",
    "dir_hu": "/hippunfold_v1.3.0/hippunfold",
    "dir_zb": "/DM_zb_37comp",
    "study": "3T",
    "ID_ctrl" : ["HC"], # patterns for control IDs in demographics file
    "ID_Pt" : ["PX"] # patterns for patient IDs in demographics file
    }

PNI = {
    "name": "PNI",
    "dir_root": "/data/mica3/BIDS_PNI",
    "dir_raw": "/rawdata",
    "dir_deriv": "/derivatives",
    "dir_mp": "/micapipe_v0.2.0",
    "dir_hu": "/hippunfold_v1.3.0/hippunfold",
    "dir_zb": "/DM_zb_37comp",
    "study": "7T",
    "ID_col" : ["PNC", "Pilot"], # column for ID in demographics file
    }

studies = [MICs, PNI]

ctrl_grp = {'ctrl' : ['CTRL']}

px_grps = { # specify patient group labels to compare to controls
    'allPX' : ['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'],
    'TLE' : ['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'],
    'TLE_L': ['TLE_L', 'mTLE_L', 'bTLE_L'],
    'TLE_R': ['TLE_R', 'mTLE_R', 'bTLE_R'],
    'FCD' : ['FLE_R', 'FLE_L'],
    'MFCL' : ['MFCL', 'bTLE'],
    'UKN' : ['UKN_L', 'UKN_U']
}

# Make list of dict items for group definitions
groups = [
    {'TLE_L': px_grps['TLE_L']},
    {'TLE_R': px_grps['TLE_R']},
    ctrl_grp
]

ses_specs  = { # session comparions ses_specs. Includes paths, what maps to run, statistics to compute, moderators, etc. NOTE. All moderators listed should be in demographics dictionary
    'prjDir_root' : "/host/verges/tank/data/daniel/3T7T/z", # output directory for smoothed cortical maps
    'prjDir_outs' : "/outputs/btwSES",
    'prjDir_out_figs': "/outputs/btwSES/figures",
    'prjDir_maps' : "/maps",
    
    'ctx': True, # whether to include cortical analyses
    'surf_ctx': ['fsLR-5k'],
    'lbl_ctx': ['midthickness', 'pial', 'white', 'swm1.0mm'], # pial, midthick, white, etc
    'ft_ctx': ['thickness', 'T1map', 'flair', 'ADC', 'FA'], # features: T1map, flair, thickness, FA, ADC
    'smth_ctx': [5, 10], # in mm
    
    'hipp': True, # whether to include hippocampal analyses
    'surf_hipp': ['0p5mm'],
    'lbl_hipp': ['midthickness', "inner", "outer"], # outer, inner, midthickness, etc
    'ft_hipp': ['thickness', 'T1map', 'flair', 'ADC', 'FA'], # features: T1map, flair, thickness, FA, ADC
    'smth_hipp': [2, 5], # in mm
    
    # within study comparisons
    'col_grp': 'grp_detailed',  # column in df_demo with group labels
    'cor': True,
    'moderators': [demographics['age'], demographics['sex'], demographics['date'], demographics['grp'], 'Seizure onset (yr)'],
}



In [95]:
importlib.reload(demo)
reimport = True
if 'df_demo' not in locals() or df_demo is None or reimport:
    df_demo = pd.read_csv(demographics['pth'])
    print(f"df_demo loaded from {demographics['pth']}")


df_demo[demographics['date']] = pd.to_datetime(df_demo[demographics['date']], errors='coerce', dayfirst=True) # format the date column
df_demo['ID_study'] = df_demo.apply(lambda row: row[demographics['ID_3T']] if row['study'] == '3T' else (row[demographics['ID_7T']] if row['study'] == '7T' else np.nan), axis=1) # create column assigning appropriate study ID for that row
demographics['col_studyID'] = 'ID_study' # add to demographics dict
print(df_demo[['UID','MICS_ID', 'PNI_ID', 'study', 'ID_study', 'SES', 'Date', 'grp_detailed']])
df_demo.columns

df_demo loaded from /host/verges/tank/data/daniel/3T7T/z/outputs/03b_demoPths_clean_16Sep2025-104025.csv
         UID MICS_ID    PNI_ID study  ID_study SES       Date                                                                             grp_detailed
0    UID0001   HC129  Pilot013    7T  Pilot013  05 2024-04-18                                                                                     CTRL
1    UID0001   HC129  Pilot013    3T     HC129  01 2024-07-09                                                                                     CTRL
2    UID0002   HC082    PNC003    7T    PNC003  01 2022-05-06                                                                                     CTRL
3    UID0002   HC082    PNC003    7T    PNC003  03 2023-03-13                                                                                     CTRL
4    UID0002   HC082    PNC003    7T    PNC003  02 2022-06-13                                                                               

Index(['UID', 'MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'lastSeizure',
       'handedness', 'education', 'sex', 'gender', 'ethnicity', 'employment',
       'dob', 'language', 'Histopatholgy',
       'Engel classification (seizure outcomes at the 6 month )',
       '# of ASMs prior current EMU admission',
       'Epilepsy classification:Focal,Generalized',
       'Genetic test (year,results)', 'Duration of admission',
       'Epilepsy diagnosis based on ILAE', '# of ASM on admission',
       'ILAE outcome after surgical resection by 1 yr',
       'Risk factors for epilepsy',
       'Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)',
       'F

In [96]:
# SUMMARY OF PARTICIPANTS/<TABLE 1>
demo.grp_summary(df_demo, col_grp='grp_detailed', save_pth=None)
print("-"*100)
print("MEDIAN AGE by group")
df_demo.groupby(['grp_detailed', 'study'])['age'].median().sort_index(level='grp_detailed')

----------------------------------------
                                                                                         num_px  num_ses_3T  num_ses_7T max_ses_3T max_ses_7T median_ses_3T median_ses_7T
TOTAL                                                                                        60          91         102                                                  
CTRL                                                                                         19          38          56          4          6           2.0           4.0
TLE_R                                                                                         9          14           9          4          1           1.0           1.0
TLE_L                                                                                         7           7           8          1          2           1.0           1.0
FLE_L                                                                                         4           5  

grp_detailed                                                                             study
CTRL                                                                                     3T       28.544832
                                                                                         7T       27.274470
FLE_L                                                                                    3T       24.906229
                                                                                         7T       25.407255
FLE_R                                                                                    3T       33.519507
                                                                                         7T       34.212183
MFCL                                                                                     3T       34.135524
                                                                                         7T       34.214921
PATTERN NOT RECOGNIZED: lobe=IGE, lat=IGE

# ANALYSIS ses_specs

# 4. Analysis

In [97]:
# create a column in df_demo that provides a session number per participant. Should order by scan date
# each UID-study combination should have a session number starting at 1 and increasing by 1 for each additional session, ordered by date
df_demo = df_demo.sort_values(by=['UID', 'study', 'Date'])
df_demo['ses_number'] = df_demo.groupby(['UID', 'study']).cumcount() + 1
demographics['col_sesNum'] = 'ses_number' # add to demographics dict for later use
#pd.set_option('display.max_rows', None)
df_demo[['UID', 'study', 'Date', 'ses_number']].loc[0:25,:]

# create a table counting  the number of sessions per study. this should return a n_unique_studies by n_unique_sessions table
df_demo_sessions = df_demo.groupby(['study', 'ses_number']).size().unstack(fill_value=0)
df_demo_sessions

ses_number,1,2,3,4,5,6
study,,,,,,
3T,60,20,8,3,0,0
7T,60,16,10,10,5,1


In [110]:
# Iterate through specifications and session number, read in maps and keep df_demo with rows only from the session. The index of the df_maps should be 'UID'_'STUDY'_'STUDYID'_'SES'ArithmeticError
importlib.reload(tsutil)

save = True
save_pth = f"{['prjDir_root']}{ses_specs['prjDir_outs']}"
save_name = "01a_maps"
test = False
verbose = True

print(f"Creating a dictionary items with session number as an iterated item rather than study (each combo of: ses_number-feature-label-surface-smoothing).\n\tNote. Not seperating participant groups yet.")

dl = tsutil.extractMap_SES(df_mapPaths = df_demo, col_sesNum = demographics['col_sesNum'], col_studyID = demographics['col_studyID'],
                            coi = ses_specs['moderators'], save_pth = f"{ses_specs['prjDir_root']}{ses_specs['prjDir_outs']}", 
                            save_name = save_name, verbose = verbose, test=test)


Creating a dictionary items with session number as an iterated item rather than study (each combo of: ses_number-feature-label-surface-smoothing).
	Note. Not seperating participant groups yet.
36 map columns found: 18 L | 18 R.

36 map columns found.

	Processing surf-fsLR-5k_label-thickness_unsmth... (cols: ctx_hemi-L_surf-fsLR-5k_label-thickness_unsmth ctx_hemi-R_surf-fsLR-5k_label-thickness_unsmth) | region: cortex

	Processing surf-fsLR-5k_label-thickness_unsmth... (cols: ctx_hemi-L_surf-fsLR-5k_label-thickness_unsmth ctx_hemi-R_surf-fsLR-5k_label-thickness_unsmth)
		6 rows removed due to missing values for these maps. [193 rows before, 187 rows remain]
		82 unique study-IDs removed given data for 0 or 1 sessions. [187 rows before, 105 rows remain]
		34 unique study-ID pairs remain after removing 86 IDs due to having one or zero sessions with data for this column.
		sutdy-IDs removed: ['HC013', 'HC116', 'HC128', 'HC129', 'HC132', 'HC140', 'PNC029', 'PNC037', 'PNC038', 'PNC039', 'PN

In [ ]:
tsutil.print_dict(dl)


 Dict list length (96 items)

[0]
	Keys: ['sesNum', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	sesNum: 1
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: NA
	df_demo: <DataFrame shape=(34, 12)>
	df_maps: <DataFrame shape=(34, 9684)>

[1]
	Keys: ['sesNum', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	sesNum: 2
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: NA
	df_demo: <DataFrame shape=(34, 12)>
	df_maps: <DataFrame shape=(34, 9684)>

[2]
	Keys: ['sesNum', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	sesNum: 3
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: NA
	df_demo: <DataFrame shape=(18, 12)>
	df_maps: <DataFrame shape=(18, 9684)>

[3]
	Keys: ['sesNum', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	sesNum: 4
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: NA
	df_demo: <DataFra

In [ ]:
# SHOW MAP MATRICES # TODO. adatpt for sessions
importlib.reload(tsutil)

# create pngs
fig_dir = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/maps_SES/raw"
tsutil.plotMatrices(dl = dl, key = 'df_maps', sessions = [1,2], show=False, save_pth=fig_dir, test=False) # Visualize unsmoothed maps
tsutil.pngs2pdf(fig_dir, output="/host/verges/tank/data/daniel/3T7T/z/outputs/figs/maps_SES") # group pngs of same comparisons with different smoothing to single pdf

print("Should visually inspect maps, identifying feature-ID-SES combinations that are outliers. Mark for removal [editing <path/to/file name.xlsx> and rerun from step 3.")

Plotting matrices for df_maps...
[get_pair] WARNING: Multiple matches found for index 0 with keys ['region', 'surf', 'label', 'feature', 'smth']. Returning all matches: [1, 2, 3, 4, 5]
	Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/figs/maps_SES/raw/cortex_thickness_fsLR-5k_thickness_smth-NAmm_ses-12_19Sep2025-110558.png
[get_pair] WARNING: Multiple matches found for index 6 with keys ['region', 'surf', 'label', 'feature', 'smth']. Returning all matches: [7, 8, 9, 10, 11]
	Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/figs/maps_SES/raw/cortex_thickness_fsLR-5k_thickness_smth-5mm_ses-12_19Sep2025-110604.png
[get_pair] WARNING: Multiple matches found for index 12 with keys ['region', 'surf', 'label', 'feature', 'smth']. Returning all matches: [13, 14, 15, 16, 17]
	Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/figs/maps_SES/raw/cortex_thickness_fsLR-5k_thickness_smth-10mm_ses-12_19Sep2025-110609.png
[get_pair] WARNING: Multiple matches found for index 18 with keys ['regi

# Within study, vertex-wise statistics (z-, w- scores)
- compares _all_ participants to controls 


In [ ]:
# import smmothed maps
reimport = False
test = False
toPrint = True
save_name = "05a_stats_winStudy"

if 'dl' not in globals() or dl is None or reimport == True:
    pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/btwSES/01a_maps_19Sep2025-102425.pkl"
    dl = tsutil.loadPickle(pth, dlPrint=toPrint)
    
tsutil.print_dict(dl, df_print=False)

# calculate statistics
dl_winComp = tsutil.winComp(dl = dl, demographics = demographics, ctrl_grp = ctrl_grp, z = ses_specs['z'], w = ses_specs['w'], 
                            covars = ses_specs['covars'], col_grp = ses_specs['col_grp'],
                            save = True, save_pth = ses_specs['prjDir_root'] + ses_specs['prjDir_outs'], save_name = save_name,
                            verbose = True, dlPrint = True, test=test)

Loaded smoothed maps from /host/verges/tank/data/daniel/3T7T/z/outputs/04b_dl_maps_smth_17Sep2025-091726.pkl

 Dict list length (120 items)

[0]
	Keys: ['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	study: MICs
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: 5
	df_demo: <DataFrame shape=(56, 221)>
	df_maps: <DataFrame shape=(56, 9684)>

[1]
	Keys: ['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	study: PNI
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: 5
	df_demo: <DataFrame shape=(56, 221)>
	df_maps: <DataFrame shape=(56, 9684)>

[2]
	Keys: ['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	study: MICs
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: 10
	df_demo: <DataFrame shape=(56, 221)>
	df_maps: <DataFrame shape=(56, 9684)>

[3]
	Keys: ['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_map

In [7]:
importlib.reload(tsutil)
fig_dir = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/05a_winComp/raw"
tsutil.plotMatrices(dl = dl_winComp, key = 'df_z', name_append="stat-z", show=False, save_pth=fig_dir, test=False) # visualize z score maps
tsutil.plotMatrices(dl = dl_winComp, key = 'df_w', name_append="stat-w", show=False, save_pth=fig_dir, test=False) # visualize w score maps
tsutil.pngs2pdf(fig_dir, output="/host/verges/tank/data/daniel/3T7T/z/outputs/figs/05a_winComp", verbose = True) # group pngs of same comparisons with different smoothing to single pdf

Plotting matrices for df_z...
	NaN values present [df_z MICs [idx: 0]: thickness]
	NaN values present [df_z PNI [idx: 1]: thickness]
	Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/figs/05a_winComp/raw/cortex_thickness_fsLR-5k_thickness_smth-5mm_stat-z_17Sep2025-163632.png
	Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/figs/05a_winComp/raw/cortex_thickness_fsLR-5k_thickness_smth-10mm_stat-z_17Sep2025-163639.png
	Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/figs/05a_winComp/raw/cortex_T1map_fsLR-5k_midthickness_smth-5mm_stat-z_17Sep2025-163646.png
[MICs] - cortex: T1map, fsLR-5k, pial, 5mm
[PNI] - cortex: T1map, fsLR-5k, pial, 5mm
	WARNING. Missing key 'df_z'. Skipping None and None

	Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/figs/05a_winComp/raw/cortex_T1map_fsLR-5k_white_smth-5mm_stat-z_17Sep2025-163653.png
[MICs] - cortex: T1map, fsLR-5k, swm1.0mm, 5mm
[PNI] - cortex: T1map, fsLR-5k, swm1.0mm, 5mm
	WARNING. Missing key 'df_z'. Skipping None and None

	Save

# Select group of interest and ipsi/contra flip

In [ ]:
# Create new dictionary list based on previous dl.
# New dl will have the same number of dictionary items (one for each study, ft, label, surf, smth, region combination).
#   Keys of each dictionary items may change. One df for each combination of [group[len(goi)] x lateralization[_R, _L, _ic] + 1 (ctrl)] x stat[<_z>, <_w>]] 
#   If df_{stat} is none, nothing regarding this statistic will be added to dict item.

importlib.reload(tsutil)

reimport = False
test = False
toPrint = False
goi = ["TLE"] # group(s) of interest. Store main diagnosis abrev in list to allow for multiple groups

# import
pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/05a_stats_winStudy_17Sep2025-163602.pkl"
if 'dl_winComp' not in globals(): reimport = True
elif dl_winComp is None: reimport = True
if reimport: dl_winComp = tsutil.loadPickle(pth, dlPrint=toPrint)

dl_grp_ic = tsutil.grp_flip(dl = dl_winComp, demographics = demographics, 
                    goi = goi, col_grp = ses_specs['col_grp'],
                    save_pth = ses_specs['prjDir_root'] + ses_specs['prjDir_outs'], save_name = "05b_stats_winStudy_grp", test=test, verbose=verbose, dlPrint=True)


[grp_flip] Saving log to: /host/verges/tank/data/daniel/3T7T/z/outputs/05b_stats_winStudy_grp_log_18Sep2025-125307.txt
----------------------------------------------------------------------------------------------------

 Dict list length (120 items)

[0]
	Keys: ['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps', 'ctrl_IDs', 'df_w_models', 'TLE_IDs_R', 'TLE_IDs_L', 'df_z_TLE_R', 'df_z_TLE_L', 'df_z_TLE_ic', 'df_z_ctrl', 'df_w_TLE_R', 'df_w_TLE_L', 'df_w_TLE_ic', 'df_w_ctrl']
	study: MICs
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: 5
	df_demo: <DataFrame shape=(56, 221)>
	df_maps: <DataFrame shape=(56, 9684)>
	ctrl_IDs: Index(['UID0002_HC082_01', 'UID0003_HC081_01', 'UID0004_HC135_01',
       'UID0005_HC083_01', 'UID0006_HC076_01', 'UID0007_HC062_01',
       'UID0008_HC132_01', 'UID0009_HC088_01', 'UID0010_HC128_01',
       'UID0011_HC130_01', 'UID0012_HC131_01', 'UID0013_HC013_01',
       'UID0015_HC116_01', 'UID0016_HC140_01', 

# Within study Cohen's D

In [ ]:
importlib.reload(tsutil)

reimport = False
test = False
toPrint = False
save = True

stats = []
if ses_specs['z']: # statistics to compute d-scores for
    stats.append('z')
if ses_specs['w']:
    stats.append('w')

# import
pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/05b_stats_winStudy_grp_18Sep2025-125541.pkl"
if 'dl_grp_ic' not in globals(): reimport = True
elif dl_grp_ic is None: reimport = True
if reimport: dl_grp_ic = tsutil.loadPickle(pth, dlPrint=toPrint)

winD = tsutil.winD(dl = dl_grp_ic, stats = stats, ipsiTo = ses_specs.get('ipsiTo', 'L'), 
                   save = save, save_pth = ses_specs['prjDir_root'] + ses_specs['prjDir_outs'], save_name = "05c_stats_winD",
                   verbose = verbose, test = test, dlPrint = toPrint)


[winD] Saving log to: /host/verges/tank/data/daniel/3T7T/z/outputs/05c_stats_winD_log_18Sep2025-154003.txt
Start time: 2025-09-18 15:40:03
Saved dictlist with groups and ipsi/contra statistics dfs to /host/verges/tank/data/daniel/3T7T/z/outputs/05c_stats_winD_2025Sep18-154003.pkl

Completed. End time: 2025-09-18 16:01:08
Duration: 21:4 (mm:ss)


In [ ]:
# visualize matrices
importlib.reload(tsutil)
save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/05c_winD/raw"
tsutil.plotMatrices(dl = winD[83], key = 'df_d', show=False, save_pth=save_pth) # Visualize unsmoothed maps
tsutil.plotMatrices(dl = winD[83], key = 'df_d_ic', show=False, save_pth=save_pth) # Visualize unsmoothed maps
tsutil.pngs2pdf(fig_dir = save_pth, output = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/05c_winD", verbose = True)

Plotting matrices for df_d...
	WARNING. No matching index found for: [PNI] - hippocampus: T1map, 0p5mm, outer, 5mm (idx 0).
Skipping.
Plotting matrices for df_d_ic...
	WARNING. No matching index found for: [PNI] - hippocampus: T1map, 0p5mm, outer, 5mm (idx 0).
Skipping.


# Between study: D-score differences
- Identify pairs of dictionary items
- Extract d scoring statitics and compute:
- raw d dif
- d dif / ctrl d

In [ ]:
importlib.reload(tsutil)

reimport = False
test = False
toPrint = False
verbose = True

# import 
pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/05c_stats_winD_2025Sep18-154003.pkl"
if 'winD' not in globals(): reimport = True
elif winD is None: reimport = True
if reimport: dl = tsutil.loadPickle(pth, dlPrint=toPrint)

comps = tsutil.btwD(dl = winD,
                    save = save, save_pth = ses_specs['prjDir_root'] + ses_specs['prjDir_outs'], save_name = "05d_stats_btwStudy",
                    verbose = verbose, test = test, dlPrint = toPrint)


[winD] Saving log to: /host/verges/tank/data/daniel/3T7T/z/outputs/05d_stats_btwStudy_log_18Sep2025-161131.txt
Start time: 2025-09-18 16:11:31
Saved dictlist with groups and ipsi/contra statistics dfs to /host/verges/tank/data/daniel/3T7T/z/outputs/05d_stats_btwStudy_18Sep2025-161131.pkl

Completed btwD. End time: 2025-09-18 16:11:52
Duration: 00:20 (mm:ss)


# Visualize